In [ ]:
!pip install scipy==1.1.0

In [ ]:
import numpy as np
import os
from scipy.misc.pilutil import imresize
from scipy.misc.pilutil import imread
import datetime
import os
import warnings
warnings.filterwarnings("ignore")
import abc
from sys import getsizeof

In [ ]:
import imageio

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
#tf.set_random_seed(30)
tf.random.set_seed(30)

Using TensorFlow backend.


In [ ]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow import keras

### Importing tensorflow.keras throughout as earlier I was facing issues with compatibility between tensorflow.keras and just keras.  

In [ ]:
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
#from tensorflow.keras.layers.convolutional import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model

In [ ]:
!unzip Project_data.zip

Archive:  Project_data.zip
   creating: Project_data/
   creating: Project_data/train/
  inflating: Project_data/train.csv  
   creating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00012.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00014.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00016.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00018.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00020.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00022.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00024.png  
  inflating: Pro

In [ ]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

# Copying the best performing model [Solution 6 with batch size 40, mentioned as experiment 6c in write up doc.] at the top.

## Choosing this as the final model given similar performance to 2nd best LSTM_GRU model with 35% less trainable parameters, i.e. less training required and also requires lower memory to live. With slight tradeoff in terms of overfitting.

### To check out models built with starter code/generators, please refer to appendix, right after this final model.

# Solution 6: Given resizing image, altering number of frames, normalization, usage of cnn+rnn did not give significant boost to accuracy, using custom data augmentation techniques.

### 1. Using an augmented generator function.
### 2. Image resized to: 120 x 120
### 3. Number of frames used: 30
### 4. Epochs: 10 , Number of steps per Epoch: 67, Batch size: 10
### 5. Normalization factor: Division by 127.5
### 6. LSTM/RNN: No
### 7. Image resizing method: Intercubic
### 8. Transformation applied: Affine
### 9. Training data = original training data + augmented training data

For details on image resizing refer: https://www.geeksforgeeks.org/image-resizing-using-opencv-python/

### Model parameters

In [ ]:
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # length of the video frames
nb_channel = 3  # no. of channels
batch_size = 40 # batch size

### Writing some custom transformation functions (affine and normalisation)

In [ ]:
# Generate random affine transform on image
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [ ]:
# Function to normalise the data
def normalize_data(data):
    return data/127.5-1

### Batch initialisation function

In [ ]:
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel))
    batch_labels = np.zeros((batch_size,5))
    return batch_data, batch_labels

### Writing custom generator functions.

In [ ]:
def train_load_batch_images(source_path, folder_list, batch_num, batch_size, t):

    batch_data,batch_labels = init_batch_data(batch_size)
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)
    img_idx = [x for x in range(0, nb_frames)]

    for folder in range(batch_size):
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0]))
        # Generate a random affine to be used in image transformation for buidling augmented data set
        M = get_random_affine()


        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #given imread reads an image in opencv'2 default BGR format, converting it back into RGB.

            resized = cv2.resize(image, (nb_cols,nb_rows), interpolation = cv2.INTER_CUBIC)
            #Notice how nb_rows and nb_cols are flipped in resize function,
            #this is because tuple passed for determining the size of new image follows the order (width, height) unlike as expected (height, width).
            batch_data[folder,idx] = (resized)
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1

    batch_data = np.append(batch_data, batch_data_aug, axis = 0)
    batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0)
    return batch_data, batch_labels

In [ ]:

def val_load_batch_images(source_path, folder_list, batch_num, batch_size, t):

    batch_data,batch_labels = init_batch_data(batch_size)
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)

    img_idx = [x for x in range(0, nb_frames)]

    for folder in range(batch_size):
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0]))
        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(image, (nb_cols,nb_rows), interpolation = cv2.INTER_CUBIC)
            batch_data[folder,idx] = (resized)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1

    return batch_data, batch_labels

In [ ]:
def train_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            yield train_load_batch_images(source_path, folder_list, batch, batch_size, t)

        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield train_load_batch_images(source_path, folder_list, num_batches, batch_size, t)

In [ ]:
def val_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            yield val_load_batch_images(source_path, folder_list, batch, batch_size, t)

        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield val_load_batch_images(source_path, folder_list, num_batches, batch_size, t)

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


### Writing model

In [ ]:
nb_filters = [8,16,32,64]
#nb_dense = [1000, 500, 5]
nb_dense = [256, 128, 5]
# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(layers.Conv3D(nb_filters[0], kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[1], kernel_size=(3,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[2], kernel_size=(1,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[3], kernel_size=(1,3,3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.25))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(layers.Flatten())

model.add(layers.Dense(nb_dense[0], activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(nb_dense[1], activation='relu'))
model.add(layers.Dropout(0.5))

#softmax layer
model.add(layers.Dense(nb_dense[2], activation='softmax'))

### Compiling model

In [ ]:
optimiser = keras.optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation (Activation)      (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 60, 60, 16)    0

In [ ]:
train_generator = train_generator(train_path, train_doc, batch_size)
val_generator = val_generator(val_path, val_doc, batch_size)

### Creating model save checkpoints

In [ ]:
model_name = 'model_' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

W0209 13:10:45.107136 139790279550784 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

## Experimenting with batch size = 40

In [ ]:
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/train ; batch size = 40
Epoch 1/30
17/17 [==============================] - ETA: 0s - loss: 0.4138 - categorical_accuracy: 0.8499Source path =  ./Project_data/val ; batch size = 40

Epoch 00001: saving model to model_init_2021-02-0811_00_13.463987/model-00001-0.41381-0.84992-0.64266-0.76000.h5
17/17 [==============================] - 78s 5s/step - loss: 0.4138 - categorical_accuracy: 0.8499 - val_loss: 0.6427 - val_categorical_accuracy: 0.7600 - lr: 1.5625e-05
Epoch 2/30
17/17 [==============================] - ETA: 0s - loss: 0.4827 - categorical_accuracy: 0.8120
Epoch 00002: saving model to model_init_2021-02-0811_00_13.463987/model-00002-0.48265-0.81202-0.61462-0.76667.h5
17/17 [==============================] - 47s 3s/step - loss: 0.4827 - categorical_accuracy: 0.8120 - val_loss: 0.6146 - val_categorical_accuracy: 0.7667 - lr: 1.5625e-05
Epoch 3/30
17/17 [==============================] - ETA: 0s - loss: 0.4206 - categorical_accuracy: 0.8542
Epoch 0000

# Appendix of all the solution approaches.

# Solution 1, details:

### 1. Using a vanila generator function without any data augmentation.
### 2. Image resized to: 100 x 100
### 3. Number of frames used: 30
### 4. Epochs: 10 , Number of steps per Epoch: 67, Batch size: 10
### 5. Normalization: division by 127.5
### 6. LSTM/RNN: No

#### Declaring parameters to be used for solution 1

In [ ]:
nb_frames = 30
nb_rows = 100
nb_cols = 100
nb_channel = 3
batch_size = 10
user_def_num_epochs = 10

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = user_def_num_epochs # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


# Generator

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(source_path)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,30,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(100, 100)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 127.5
                    batch_data[folder,idx,:,:,1] /= 127.5 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 127.5 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        if((len(source_path)%batch_size)//2==0):
            batch_size = 2
        else:
            batch_size = 1
        num_batches = len(source_path)%batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,30,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(100, 100)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 127.5
                    batch_data[folder,idx,:,:,1] /= 127.5 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 127.5 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

# Model architecture:  Conv3d without LSTM

In [ ]:
nb_filters = [8,16,32,64]
#nb_dense = [1000, 500, 5]
nb_dense = [256, 128, 5]
# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(layers.Conv3D(nb_filters[0], kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[1], kernel_size=(3,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[2], kernel_size=(1,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[3], kernel_size=(1,3,3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.25))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(layers.Flatten())

model.add(layers.Dense(nb_dense[0], activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(nb_dense[1], activation='relu'))
model.add(layers.Dropout(0.5))

#softmax layer
model.add(layers.Dense(nb_dense[2], activation='softmax'))

In [ ]:
optimiser = keras.optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 100, 100, 8)   32        
_________________________________________________________________
activation (Activation)      (None, 30, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 50, 50, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 50, 50, 16)    3472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 50, 50, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 50, 50, 16)    0

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'Gesture_recog_v1_0' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)


LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
#LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,verbose = 0, mode = "auto",epsilon = 1e-04, cooldown = 0,min_lr = 0)

callbacks_list = [checkpoint, LR]

W0208 13:58:50.965767 140224354600768 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0208 13:58:54.211295 140224354600768 deprecation.py:323] From <ipython-input-17-2787df387a23>:4: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.


Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6089 - categorical_accuracy: 0.1935Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to Gesture_recog_v1_0_2021-02-0813_58_18.934428/model-00001-1.60893-0.19355-1.60683-0.31579.h5
67/67 [==============================] - 28s 419ms/step - loss: 1.6089 - categorical_accuracy: 0.1935 - val_loss: 1.6068 - val_categorical_accuracy: 0.3158 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6070 - categorical_accuracy: 0.2388
Epoch 00002: saving model to Gesture_recog_v1_0_2021-02-0813_58_18.934428/model-00002-1.60698-0.23881-1.60517-0.21429.h5
67/67 [==============================] - 27s 408ms/step - loss: 1.6070 - categorical_accuracy: 0.2388 - val_loss: 1.6052 - val_categorical_accuracy: 0.2143 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6093 - categorical_accuracy: 0.1567


# Solution 2, details:

### 1. Using a vanila generator function without any data augmentation.
### 2. Image resized to: 120 x 120
### 3. Number of frames used: 30
### 4. Epochs: 10 , Number of steps per Epoch: 67, Batch size: 10
### 5. Normalization: division by 127.5
### 6. LSTM/RNN: No

In [ ]:
nb_frames = 30
nb_rows = 120
nb_cols = 120
nb_channel = 3
batch_size = 10
user_def_num_epochs = 10

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = user_def_num_epochs # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(source_path)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,30,nb_rows,nb_cols,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(nb_rows, nb_cols)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 127.5
                    batch_data[folder,idx,:,:,1] /= 127.5 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 127.5 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        if((len(source_path)%batch_size)//2==0):
            batch_size = 2
        else:
            batch_size = 1
        num_batches = len(source_path)%batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,30,nb_rows,nb_cols,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(nb_rows, nb_cols)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 127.5
                    batch_data[folder,idx,:,:,1] /= 127.5 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 127.5 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [ ]:
nb_filters = [8,16,32,64]
#nb_dense = [1000, 500, 5]
nb_dense = [256, 128, 5]
# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(layers.Conv3D(nb_filters[0], kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[1], kernel_size=(3,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[2], kernel_size=(1,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[3], kernel_size=(1,3,3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.25))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(layers.Flatten())

model.add(layers.Dense(nb_dense[0], activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(nb_dense[1], activation='relu'))
model.add(layers.Dropout(0.5))

#softmax layer
model.add(layers.Dense(nb_dense[2], activation='softmax'))

In [ ]:
optimiser = keras.optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation (Activation)      (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 60, 60, 16)    0

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'Gesture_recog_solution2' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)


LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
#LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,verbose = 0, mode = "auto",epsilon = 1e-04, cooldown = 0,min_lr = 0)

callbacks_list = [checkpoint, LR]

W0208 13:26:26.022702 140334861158208 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0208 13:26:28.708269 140334861158208 deprecation.py:323] From <ipython-input-20-2787df387a23>:4: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.


Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6089 - categorical_accuracy: 0.1935Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to Gesture_recog_solution2_2021-02-0813_26_18.651421/model-00001-1.60893-0.19355-1.60683-0.31579.h5
67/67 [==============================] - 112s 2s/step - loss: 1.6089 - categorical_accuracy: 0.1935 - val_loss: 1.6068 - val_categorical_accuracy: 0.3158 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6070 - categorical_accuracy: 0.2388
Epoch 00002: saving model to Gesture_recog_solution2_2021-02-0813_26_18.651421/model-00002-1.60698-0.23881-1.60517-0.21429.h5
67/67 [==============================] - 124s 2s/step - loss: 1.6070 - categorical_accuracy: 0.2388 - val_loss: 1.6052 - val_categorical_accuracy: 0.2143 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6093 - categorical_accuracy: 0

# Solution 3, details:

### 1. Using a vanila generator function without any data augmentation.
### 2. Image resized to: 120 x 120
### 3. Number of frames used: 20
### 4. Epochs: 10 , Number of steps per Epoch: 67, Batch size: 10
### 5. Normalization: division by 127.5
### 6. LSTM/RNN: No

In [ ]:
from PIL import Image

In [ ]:
nb_frames = 20
nb_rows = 120
nb_cols = 120
nb_channel = 3
batch_size = 10
user_def_num_epochs = 10

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = user_def_num_epochs # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(source_path)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(nb_rows, nb_cols)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 127.5
                    batch_data[folder,idx,:,:,1] /= 127.5 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 127.5 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        if((len(source_path)%batch_size)//2==0):
            batch_size = 2
        else:
            batch_size = 1
        num_batches = len(source_path)%batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(nb_rows, nb_cols)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 127.5
                    batch_data[folder,idx,:,:,1] /= 127.5 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 127.5 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [ ]:
nb_filters = [8,16,32,64]
#nb_dense = [1000, 500, 5]
nb_dense = [256, 128, 5]
# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(layers.Conv3D(nb_filters[0], kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[1], kernel_size=(3,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[2], kernel_size=(1,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[3], kernel_size=(1,3,3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.25))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(layers.Flatten())

model.add(layers.Dense(nb_dense[0], activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(nb_dense[1], activation='relu'))
model.add(layers.Dropout(0.5))

#softmax layer
model.add(layers.Dense(nb_dense[2], activation='softmax'))

In [ ]:
optimiser = keras.optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_32 (Conv3D)           (None, 20, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_24 (Batc (None, 20, 120, 120, 8)   32        
_________________________________________________________________
activation_32 (Activation)   (None, 20, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_32 (MaxPooling (None, 10, 60, 60, 8)     0         
_________________________________________________________________
conv3d_33 (Conv3D)           (None, 10, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_25 (Batc (None, 10, 60, 60, 16)    64        
_________________________________________________________________
activation_33 (Activation)   (None, 10, 60, 60, 16)   

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'Gesture_recog_solution2' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)


LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
#LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,verbose = 0, mode = "auto",epsilon = 1e-04, cooldown = 0,min_lr = 0)

callbacks_list = [checkpoint, LR]

W0208 14:35:15.559838 140224354600768 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6089 - categorical_accuracy: 0.1935Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to Gesture_recog_solution2_2021-02-0814_19_17.898560/model-00001-1.60893-0.19355-1.60683-0.31579.h5
67/67 [==============================] - 8s 123ms/step - loss: 1.6089 - categorical_accuracy: 0.1935 - val_loss: 1.6068 - val_categorical_accuracy: 0.3158 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6070 - categorical_accuracy: 0.2388
Epoch 00002: saving model to Gesture_recog_solution2_2021-02-0814_19_17.898560/model-00002-1.60698-0.23881-1.60517-0.21429.h5
67/67 [==============================] - 8s 117ms/step - loss: 1.6070 - categorical_accuracy: 0.2388 - val_loss: 1.6052 - val_categorical_accuracy: 0.2143 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6093 - categorical_accuracy:

# Solution 4, details:

### 1. Using a vanila generator function without any data augmentation.
### 2. Image resized to: 120 x 120
### 3. Number of frames used: 20
### 4. Epochs: 10 , Number of steps per Epoch: 67, Batch size: 10
### 5. Normalization: division by 255
### 6. LSTM/RNN: No

In [ ]:
nb_frames = 20
nb_rows = 120
nb_cols = 120
nb_channel = 3
batch_size = 10
user_def_num_epochs = 10

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = user_def_num_epochs # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(source_path)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(nb_rows, nb_cols)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 225
                    batch_data[folder,idx,:,:,1] /= 225 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 225 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        if((len(source_path)%batch_size)//2==0):
            batch_size = 2
        else:
            batch_size = 1
        num_batches = len(source_path)%batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(nb_rows, nb_cols)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 225
                    batch_data[folder,idx,:,:,1] /= 225 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 225 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [ ]:
nb_filters = [8,16,32,64]
#nb_dense = [1000, 500, 5]
nb_dense = [256, 128, 5]
# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(layers.Conv3D(nb_filters[0], kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[1], kernel_size=(3,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[2], kernel_size=(1,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[3], kernel_size=(1,3,3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.25))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(layers.Flatten())

model.add(layers.Dense(nb_dense[0], activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(nb_dense[1], activation='relu'))
model.add(layers.Dropout(0.5))

#softmax layer
model.add(layers.Dense(nb_dense[2], activation='softmax'))

In [ ]:
optimiser = keras.optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_24 (Conv3D)           (None, 20, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_18 (Batc (None, 20, 120, 120, 8)   32        
_________________________________________________________________
activation_24 (Activation)   (None, 20, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_24 (MaxPooling (None, 10, 60, 60, 8)     0         
_________________________________________________________________
conv3d_25 (Conv3D)           (None, 10, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_19 (Batc (None, 10, 60, 60, 16)    64        
_________________________________________________________________
activation_25 (Activation)   (None, 10, 60, 60, 16)   

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'Gesture_recog_solution4' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)


LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
#LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,verbose = 0, mode = "auto",epsilon = 1e-04, cooldown = 0,min_lr = 0)

callbacks_list = [checkpoint, LR]

W0208 14:22:36.315734 140224354600768 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6090 - categorical_accuracy: 0.2258Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to Gesture_recog_solution4_2021-02-0814_21_55.009976/model-00001-1.60904-0.22581-1.61382-0.26316.h5
67/67 [==============================] - 10s 149ms/step - loss: 1.6090 - categorical_accuracy: 0.2258 - val_loss: 1.6138 - val_categorical_accuracy: 0.2632 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6111 - categorical_accuracy: 0.1866
Epoch 00002: saving model to Gesture_recog_solution4_2021-02-0814_21_55.009976/model-00002-1.61111-0.18657-1.61058-0.14286.h5
67/67 [==============================] - 10s 147ms/step - loss: 1.6111 - categorical_accuracy: 0.1866 - val_loss: 1.6106 - val_categorical_accuracy: 0.1429 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6086 - categorical_accurac

# Solution 5, details:

### 1. Using a vanila generator function without any data augmentation.
### 2. Image resized to: 120 x 120
### 3. Number of frames used: 20
### 4. Epochs: 10 , Number of steps per Epoch: 67, Batch size: 10
### 5. Normalization: Division by 255
### 6. LSTM/RNN: Yes

In [ ]:
from tensorflow.keras.layers import Dropout, LSTM, Bidirectional

In [ ]:
nb_frames = 20
nb_rows = 120
nb_cols = 120
nb_channel = 3
batch_size = 10
user_def_num_epochs = 10

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = user_def_num_epochs # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [ ]:
def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)
    return img[starty:starty+cropy,startx:startx+cropx]

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(source_path)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(nb_rows, nb_cols)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 225
                    batch_data[folder,idx,:,:,1] /= 225 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 225 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        if((len(source_path)%batch_size)//2==0):
            batch_size = 2
        else:
            batch_size = 1
        num_batches = len(source_path)%batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(nb_rows, nb_cols)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 225
                    batch_data[folder,idx,:,:,1] /= 225 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 225 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [ ]:
nb_filters = [8,16,32,64]
#nb_dense = [1000, 500, 5]
nb_dense = [256, 128, 5]
# Input
Input_shape_1=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()
model.add(TimeDistributed(layers.Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same'), input_shape=Input_shape_1))
model.add(TimeDistributed(layers.Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Flatten()))

model.add(layers.Dropout(0.5))
model.add(layers.LSTM(512, return_sequences=False, dropout=0.5))
model.add(layers.Dense(5, activation='softmax'))

In [ ]:
optimiser = keras.optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 20, 60, 60, 32)    4736      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 58, 58, 32)    9248      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 29, 29, 32)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 29, 29, 64)    18496     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 29, 29, 64)    36928     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 14, 14, 64)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 14, 14, 128) 

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'Gesture_recog_solution2' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)


LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
#LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,verbose = 0, mode = "auto",epsilon = 1e-04, cooldown = 0,min_lr = 0)

callbacks_list = [checkpoint, LR]

W0208 15:03:21.018061 140224354600768 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6135 - categorical_accuracy: 0.1774Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to Gesture_recog_solution2_2021-02-0815_03_14.569110/model-00001-1.61347-0.17742-1.59776-0.21053.h5
67/67 [==============================] - 10s 147ms/step - loss: 1.6135 - categorical_accuracy: 0.1774 - val_loss: 1.5978 - val_categorical_accuracy: 0.2105 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6109 - categorical_accuracy: 0.2687
Epoch 00002: saving model to Gesture_recog_solution2_2021-02-0815_03_14.569110/model-00002-1.61093-0.26866-1.59999-0.28571.h5
67/67 [==============================] - 8s 124ms/step - loss: 1.6109 - categorical_accuracy: 0.2687 - val_loss: 1.6000 - val_categorical_accuracy: 0.2857 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6061 - categorical_accuracy

# Solution 6: Given resizing image, altering number of frames, normalization did not give significant boost to accuracy, using custom data augmentation techniques.

### 1. Using an augmented generator function.
### 2. Image resized to: 120 x 120
### 3. Number of frames used: 30
### 4. Epochs: 10 , Number of steps per Epoch: 67, Batch size: 10
### 5. Normalization: Division by 127.5
### 6. LSTM/RNN: No
### 7. Image resizing method: Intercubic
### 8. Transformation applied: Affine
### 9. Training data = original training data + augmented training data

For details on image resizing refer: https://www.geeksforgeeks.org/image-resizing-using-opencv-python/

In [ ]:
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # length of the video frames
nb_channel = 3  # no. of channels
batch_size = 10 # batch size

In [ ]:
# Generate random affine transform on image
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [ ]:
# Function to normalise the data
def normalize_data(data):
    return data/127.5-1

In [ ]:
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel))
    batch_labels = np.zeros((batch_size,5))
    return batch_data, batch_labels

In [ ]:
def train_load_batch_images(source_path, folder_list, batch_num, batch_size, t):

    batch_data,batch_labels = init_batch_data(batch_size)
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)
    img_idx = [x for x in range(0, nb_frames)]

    for folder in range(batch_size):
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0]))
        # Generate a random affine to be used in image transformation for buidling augmented data set
        M = get_random_affine()


        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #given imread reads an image in opencv'2 default BGR format, converting it back into RGB.

            resized = cv2.resize(image, (nb_cols,nb_rows), interpolation = cv2.INTER_CUBIC)
            #Notice how nb_rows and nb_cols are flipped in resize function,
            #this is because tuple passed for determining the size of new image follows the order (width, height) unlike as expected (height, width).
            batch_data[folder,idx] = (resized)
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1

    batch_data = np.append(batch_data, batch_data_aug, axis = 0)
    batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0)
    return batch_data, batch_labels

In [ ]:

def val_load_batch_images(source_path, folder_list, batch_num, batch_size, t):

    batch_data,batch_labels = init_batch_data(batch_size)
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)

    img_idx = [x for x in range(0, nb_frames)]

    for folder in range(batch_size):
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0]))
        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(image, (nb_cols,nb_rows), interpolation = cv2.INTER_CUBIC)
            batch_data[folder,idx] = (resized)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1

    return batch_data, batch_labels

In [ ]:
def train_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            yield train_load_batch_images(source_path, folder_list, batch, batch_size, t)

        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield train_load_batch_images(source_path, folder_list, num_batches, batch_size, t)

In [ ]:
def val_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            yield val_load_batch_images(source_path, folder_list, batch, batch_size, t)

        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield val_load_batch_images(source_path, folder_list, num_batches, batch_size, t)

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [ ]:
nb_filters = [8,16,32,64]
#nb_dense = [1000, 500, 5]
nb_dense = [256, 128, 5]
# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(layers.Conv3D(nb_filters[0], kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[1], kernel_size=(3,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[2], kernel_size=(1,3,3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

model.add(layers.Conv3D(nb_filters[3], kernel_size=(1,3,3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.25))

model.add(layers.MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(layers.Flatten())

model.add(layers.Dense(nb_dense[0], activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(nb_dense[1], activation='relu'))
model.add(layers.Dropout(0.5))

#softmax layer
model.add(layers.Dense(nb_dense[2], activation='softmax'))

In [ ]:
optimiser = keras.optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_15 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_4 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_16 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_5 (Activation)    (None, 15, 60, 60, 16)   

In [ ]:
train_generator = train_generator(train_path, train_doc, batch_size)
val_generator = val_generator(val_path, val_doc, batch_size)

In [ ]:

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

W0208 12:10:25.748686 140403264730944 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

## Experimenting with batch size = 10

In [ ]:
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0208 11:00:58.582939 140214338103104 deprecation.py:323] From <ipython-input-37-6f6c4ce25676>:5: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.


Source path =  ./Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - ETA: 0s - loss: 1.8044 - categorical_accuracy: 0.2474Source path =  ./Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-02-0811_00_13.463987/model-00001-1.80441-0.24736-1.91539-0.23000.h5
67/67 [==============================] - 94s 1s/step - loss: 1.8044 - categorical_accuracy: 0.2474 - val_loss: 1.9154 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/30
67/67 [==============================] - ETA: 0s - loss: 1.5058 - categorical_accuracy: 0.3532
Epoch 00002: saving model to model_init_2021-02-0811_00_13.463987/model-00002-1.50584-0.35323-1.50502-0.30000.h5
67/67 [==============================] - 30s 453ms/step - loss: 1.5058 - categorical_accuracy: 0.3532 - val_loss: 1.5050 - val_categorical_accuracy: 0.3000 - lr: 0.0010
Epoch 3/30
67/67 [==============================] - ETA: 0s - loss: 1.4537 - categorical_accuracy: 0.3383
Epoch 00003: sa

## Experimenting with batch size = 20

In [ ]:
batch_size = 20

train_generator = train_generator(train_path, train_doc, batch_size)
val_generator = val_generator(val_path, val_doc, batch_size)



model_name = 'experiment_test0' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]



if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0208 17:30:17.580134 140711209338688 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


Source path =  ./Project_data/train ; batch size = 20
Epoch 1/30
34/34 [==============================] - ETA: 0s - loss: 1.8583 - categorical_accuracy: 0.2474Source path =  ./Project_data/val ; batch size = 20

Epoch 00001: saving model to experiment_test0_2021-02-0817_30_05.963213/model-00001-1.85829-0.24736-2.35867-0.23000.h5
34/34 [==============================] - 82s 2s/step - loss: 1.8583 - categorical_accuracy: 0.2474 - val_loss: 2.3587 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/30
34/34 [==============================] - ETA: 0s - loss: 1.5373 - categorical_accuracy: 0.3529
Epoch 00002: saving model to experiment_test0_2021-02-0817_30_05.963213/model-00002-1.53730-0.35294-3.43864-0.24000.h5
34/34 [==============================] - 21s 624ms/step - loss: 1.5373 - categorical_accuracy: 0.3529 - val_loss: 3.4386 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 3/30
34/34 [==============================] - ETA: 0s - loss: 1.5989 - categorical_accuracy: 0.3039
Epo

## Experimenting with batch size = 40

In [ ]:
batch_size = 40

train_generator = train_generator(train_path, train_doc, batch_size)
val_generator = val_generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/train ; batch size = 40
Epoch 1/30
17/17 [==============================] - ETA: 0s - loss: 0.4138 - categorical_accuracy: 0.8499Source path =  ./Project_data/val ; batch size = 40

Epoch 00001: saving model to model_init_2021-02-0811_00_13.463987/model-00001-0.41381-0.84992-0.64266-0.76000.h5
17/17 [==============================] - 78s 5s/step - loss: 0.4138 - categorical_accuracy: 0.8499 - val_loss: 0.6427 - val_categorical_accuracy: 0.7600 - lr: 1.5625e-05
Epoch 2/30
17/17 [==============================] - ETA: 0s - loss: 0.4827 - categorical_accuracy: 0.8120
Epoch 00002: saving model to model_init_2021-02-0811_00_13.463987/model-00002-0.48265-0.81202-0.61462-0.76667.h5
17/17 [==============================] - 47s 3s/step - loss: 0.4827 - categorical_accuracy: 0.8120 - val_loss: 0.6146 - val_categorical_accuracy: 0.7667 - lr: 1.5625e-05
Epoch 3/30
17/17 [==============================] - ETA: 0s - loss: 0.4206 - categorical_accuracy: 0.8542
Epoch 0000

## Experimenting with batch size = 60

In [ ]:
batch_size = 60

train_generator = train_generator(train_path, train_doc, batch_size)
val_generator = val_generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0208 12:10:52.206804 140403264730944 deprecation.py:323] From <ipython-input-34-5e5d642fb6ad>:19: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.


Source path =  ./Project_data/train ; batch size = 60
Epoch 1/30
12/12 [==============================] - ETA: 0s - loss: 1.9900 - categorical_accuracy: 0.2345 Source path =  ./Project_data/val ; batch size = 60

Epoch 00001: saving model to model_init_2021-02-0812_09_28.324292/model-00001-1.98997-0.23454-1.60908-0.21000.h5
12/12 [==============================] - 132s 11s/step - loss: 1.9900 - categorical_accuracy: 0.2345 - val_loss: 1.6091 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 2/30
12/12 [==============================] - ETA: 0s - loss: 1.5546 - categorical_accuracy: 0.2917
Epoch 00002: saving model to model_init_2021-02-0812_09_28.324292/model-00002-1.55462-0.29167-1.70133-0.21667.h5
12/12 [==============================] - 11s 909ms/step - loss: 1.5546 - categorical_accuracy: 0.2917 - val_loss: 1.7013 - val_categorical_accuracy: 0.2167 - lr: 0.0010
Epoch 3/30
12/12 [==============================] - ETA: 0s - loss: 1.5615 - categorical_accuracy: 0.3056
Epoch 00003:

# Solution 7:

### 1. Using an augmented generator function.
### 2. Image resized to: 120 x 120
### 3. Number of frames used: 30
### 4. Epochs: 10 , Number of steps per Epoch: 67, Batch size: 10
### 5. Normalization: Division by 127.5
### 6. LSTM/RNN: Yes
### 7. Image resizing method: Intercubic
For details on image resizing refer: https://www.geeksforgeeks.org/image-resizing-using-opencv-python/

In [ ]:
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # length of the video frames
nb_channel = 3  # no. of channels
batch_size = 10 # batch size

In [ ]:
# Generate random affine transform on image
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [ ]:
# Function to normalise the data
def normalize_data(data):
    return data/127.5-1

In [ ]:
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel))
    batch_labels = np.zeros((batch_size,5))
    return batch_data, batch_labels

In [ ]:
def train_load_batch_images(source_path, folder_list, batch_num, batch_size, t):

    batch_data,batch_labels = init_batch_data(batch_size)
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)
    img_idx = [x for x in range(0, nb_frames)]

    for folder in range(batch_size):
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0]))
        # Generate a random affine to be used in image transformation for buidling augmented data set
        M = get_random_affine()


        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #given imread reads an image in opencv'2 default BGR format, converting it back into RGB.

            resized = cv2.resize(image, (nb_cols,nb_rows), interpolation = cv2.INTER_CUBIC)
            #Notice how nb_rows and nb_cols are flipped in resize function,
            #this is because tuple passed for determining the size of new image follows the order (width, height) unlike as expected (height, width).
            batch_data[folder,idx] = (resized)
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1

    batch_data = np.append(batch_data, batch_data_aug, axis = 0)
    batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0)
    return batch_data, batch_labels

In [ ]:

def val_load_batch_images(source_path, folder_list, batch_num, batch_size, t):

    batch_data,batch_labels = init_batch_data(batch_size)
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)

    img_idx = [x for x in range(0, nb_frames)]

    for folder in range(batch_size):
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0]))
        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(image, (nb_cols,nb_rows), interpolation = cv2.INTER_CUBIC)
            batch_data[folder,idx] = (resized)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1

    return batch_data, batch_labels

In [ ]:
def train_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            yield train_load_batch_images(source_path, folder_list, batch, batch_size, t)

        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield train_load_batch_images(source_path, folder_list, num_batches, batch_size, t)

In [ ]:
def val_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            yield val_load_batch_images(source_path, folder_list, batch, batch_size, t)

        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield val_load_batch_images(source_path, folder_list, num_batches, batch_size, t)

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [ ]:
nb_filters = [8,16,32,64]
#nb_dense = [1000, 500, 5]
nb_dense = [256, 128, 5]
# Input
Input_shape_1=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()
model.add(TimeDistributed(layers.Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same'), input_shape=Input_shape_1))
model.add(TimeDistributed(layers.Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Flatten()))

model.add(layers.Dropout(0.5))
model.add(layers.LSTM(512, return_sequences=False, dropout=0.5))
model.add(layers.Dense(5, activation='softmax'))

In [ ]:
optimiser = keras.optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 60, 60, 32)    4736      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 58, 58, 32)    9248      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 29, 29, 32)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 29, 29, 64)    18496     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 29, 29, 64)    36928     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 14, 14, 64)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 14, 14, 128)  

In [ ]:
train_generator = train_generator(train_path, train_doc, batch_size)
val_generator = val_generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'Gesture_recog_temptest' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)


LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
#LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,verbose = 0, mode = "auto",epsilon = 1e-04, cooldown = 0,min_lr = 0)

callbacks_list = [checkpoint, LR]

W0208 17:01:11.028934 140711209338688 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6135 - categorical_accuracy: 0.1774Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to Gesture_recog_solution2_2021-02-0815_03_14.569110/model-00001-1.61347-0.17742-1.59776-0.21053.h5
67/67 [==============================] - 10s 147ms/step - loss: 1.6135 - categorical_accuracy: 0.1774 - val_loss: 1.5978 - val_categorical_accuracy: 0.2105 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6109 - categorical_accuracy: 0.2687
Epoch 00002: saving model to Gesture_recog_solution2_2021-02-0815_03_14.569110/model-00002-1.61093-0.26866-1.59999-0.28571.h5
67/67 [==============================] - 8s 124ms/step - loss: 1.6109 - categorical_accuracy: 0.2687 - val_loss: 1.6000 - val_categorical_accuracy: 0.2857 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6061 - categorical_accuracy

# Experimenting with a Batch size of 30

# Solution 8:

### 1. Using an augmented generator function.
### 2. Image resized to: 120 x 120
### 3. Number of frames used: 30
### 4. Epochs: 10 , Number of steps per Epoch: 67, Batch size: 30
### 5. Normalization: Division by 127.5
### 6. LSTM/RNN: Yes
### 7. Image resizing method: Intercubic
For details on image resizing refer: https://www.geeksforgeeks.org/image-resizing-using-opencv-python/

In [ ]:
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # length of the video frames
nb_channel = 3  # no. of channels
batch_size = 30 # batch size

In [ ]:
# Generate random affine transform on image
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [ ]:
# Function to normalise the data
def normalize_data(data):
    return data/127.5-1

In [ ]:
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel))
    batch_labels = np.zeros((batch_size,5))
    return batch_data, batch_labels

In [ ]:
def train_load_batch_images(source_path, folder_list, batch_num, batch_size, t):

    batch_data,batch_labels = init_batch_data(batch_size)
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)
    img_idx = [x for x in range(0, nb_frames)]

    for folder in range(batch_size):
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0]))
        # Generate a random affine to be used in image transformation for buidling augmented data set
        M = get_random_affine()


        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #given imread reads an image in opencv'2 default BGR format, converting it back into RGB.

            resized = cv2.resize(image, (nb_cols,nb_rows), interpolation = cv2.INTER_CUBIC)
            #Notice how nb_rows and nb_cols are flipped in resize function,
            #this is because tuple passed for determining the size of new image follows the order (width, height) unlike as expected (height, width).
            batch_data[folder,idx] = (resized)
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1

    batch_data = np.append(batch_data, batch_data_aug, axis = 0)
    batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0)
    return batch_data, batch_labels

In [ ]:

def val_load_batch_images(source_path, folder_list, batch_num, batch_size, t):

    batch_data,batch_labels = init_batch_data(batch_size)
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)

    img_idx = [x for x in range(0, nb_frames)]

    for folder in range(batch_size):
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0]))
        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(image, (nb_cols,nb_rows), interpolation = cv2.INTER_CUBIC)
            batch_data[folder,idx] = (resized)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1

    return batch_data, batch_labels

In [ ]:
def train_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            yield train_load_batch_images(source_path, folder_list, batch, batch_size, t)

        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield train_load_batch_images(source_path, folder_list, num_batches, batch_size, t)

In [ ]:
def val_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            yield val_load_batch_images(source_path, folder_list, batch, batch_size, t)

        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield val_load_batch_images(source_path, folder_list, num_batches, batch_size, t)

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [ ]:
nb_filters = [8,16,32,64]
# Input
Input_shape_1=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()
model.add(TimeDistributed(layers.Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same'), input_shape=Input_shape_1))
model.add(TimeDistributed(layers.Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(layers.Flatten()))

model.add(layers.Dropout(0.5))
model.add(layers.LSTM(512, return_sequences=False, dropout=0.5))
model.add(layers.Dense(5, activation='softmax'))

In [ ]:
optimiser = keras.optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_16 (TimeDis (None, 30, 60, 60, 32)    4736      
_________________________________________________________________
time_distributed_17 (TimeDis (None, 30, 58, 58, 32)    9248      
_________________________________________________________________
time_distributed_18 (TimeDis (None, 30, 29, 29, 32)    0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 30, 29, 29, 64)    18496     
_________________________________________________________________
time_distributed_20 (TimeDis (None, 30, 29, 29, 64)    36928     
_________________________________________________________________
time_distributed_21 (TimeDis (None, 30, 14, 14, 64)    0         
_________________________________________________________________
time_distributed_22 (TimeDis (None, 30, 14, 14, 128)  

In [ ]:
train_generator = train_generator(train_path, train_doc, batch_size)
val_generator = val_generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'Gesture_recog_temptest' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)


LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
#LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,verbose = 0, mode = "auto",epsilon = 1e-04, cooldown = 0,min_lr = 0)

callbacks_list = [checkpoint, LR]

W0208 17:13:50.844633 140711209338688 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/train ; batch size = 30
Epoch 1/10
23/23 [==============================] - ETA: 0s - loss: 1.8394 - categorical_accuracy: 0.2081Source path =  ./Project_data/val ; batch size = 30

Epoch 00001: saving model to Gesture_recog_temptest_2021-02-0817_13_45.357945/model-00001-1.83942-0.20814-1.64639-0.24000.h5
23/23 [==============================] - 83s 4s/step - loss: 1.8394 - categorical_accuracy: 0.2081 - val_loss: 1.6464 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 2/10
23/23 [==============================] - ETA: 0s - loss: 1.8142 - categorical_accuracy: 0.1522
Epoch 00002: saving model to Gesture_recog_temptest_2021-02-0817_13_45.357945/model-00002-1.81421-0.15217-1.67346-0.17500.h5
23/23 [==============================] - 11s 467ms/step - loss: 1.8142 - categorical_accuracy: 0.1522 - val_loss: 1.6735 - val_categorical_accuracy: 0.1750 - lr: 0.0010
Epoch 3/10
23/23 [==============================] - ETA: 0s - loss: 1.7190 - categorical_accuracy

# Solution 9: [LSTM with GRU]

### 1. Using an augmented generator function.
### 2. Image resized to: 120 x 120
### 3. Number of frames used: 30
### 4. Epochs: 10 , Number of steps per Epoch: 67, Batch size: 30
### 5. Normalization: Division by 127.5
### 6. LSTM/RNN: Yes
### 7. Image resizing method: Intercubic
For details on image resizing refer: https://www.geeksforgeeks.org/image-resizing-using-opencv-python/

In [ ]:
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # length of the video frames
nb_channel = 3  # no. of channels
batch_size = 10 # batch size

In [ ]:
# Generate random affine transform on image
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [ ]:
# Function to normalise the data
def normalize_data(data):
    return data/127.5-1

In [ ]:
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel))
    batch_labels = np.zeros((batch_size,5))
    return batch_data, batch_labels

In [ ]:
def train_load_batch_images(source_path, folder_list, batch_num, batch_size, t):

    batch_data,batch_labels = init_batch_data(batch_size)
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)
    img_idx = [x for x in range(0, nb_frames)]

    for folder in range(batch_size):
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0]))
        # Generate a random affine to be used in image transformation for buidling augmented data set
        M = get_random_affine()


        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #given imread reads an image in opencv'2 default BGR format, converting it back into RGB.

            resized = cv2.resize(image, (nb_cols,nb_rows), interpolation = cv2.INTER_CUBIC)
            #Notice how nb_rows and nb_cols are flipped in resize function,
            #this is because tuple passed for determining the size of new image follows the order (width, height) unlike as expected (height, width).
            batch_data[folder,idx] = (resized)
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1

    batch_data = np.append(batch_data, batch_data_aug, axis = 0)
    batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0)
    return batch_data, batch_labels

In [ ]:

def val_load_batch_images(source_path, folder_list, batch_num, batch_size, t):

    batch_data,batch_labels = init_batch_data(batch_size)
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)

    img_idx = [x for x in range(0, nb_frames)]

    for folder in range(batch_size):
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0]))
        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(image, (nb_cols,nb_rows), interpolation = cv2.INTER_CUBIC)
            batch_data[folder,idx] = (resized)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1

    return batch_data, batch_labels

In [ ]:
def train_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            yield train_load_batch_images(source_path, folder_list, batch, batch_size, t)

        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield train_load_batch_images(source_path, folder_list, num_batches, batch_size, t)

In [ ]:
def val_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            yield val_load_batch_images(source_path, folder_list, batch, batch_size, t)

        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield val_load_batch_images(source_path, folder_list, num_batches, batch_size, t)

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [ ]:
# Input
Input_shape_1=(nb_frames,nb_rows,nb_cols,nb_channel)
lstm_cells = 64
dense_neurons = 64
dropout = 0.25

# Define model
model = Sequential()
model.add(TimeDistributed(layers.Conv2D(16, (3, 3) , padding='same', activation='relu'),input_shape=Input_shape_1))
model.add(TimeDistributed(layers.BatchNormalization()))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2))))

model.add(TimeDistributed(layers.Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(layers.BatchNormalization()))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2))))

model.add(TimeDistributed(layers.Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(layers.BatchNormalization()))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2))))

model.add(TimeDistributed(layers.Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(layers.BatchNormalization()))
model.add(TimeDistributed(layers.MaxPooling2D((2, 2))))

model.add(TimeDistributed(layers.Flatten()))

model.add(GRU(lstm_cells))
model.add(layers.Dropout(dropout))

model.add(layers.Dense(dense_neurons,activation='relu'))
model.add(layers.Dropout(dropout))

model.add(layers.Dense(5, activation='softmax'))

In [ ]:
optimiser = keras.optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 30, 30, 64)   

In [ ]:
train_generator = train_generator(train_path, train_doc, batch_size)
val_generator = val_generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'Gesture_recog_temptest' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)


LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
#LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,verbose = 0, mode = "auto",epsilon = 1e-04, cooldown = 0,min_lr = 0)

callbacks_list = [checkpoint, LR]

W0209 10:48:57.065983 139948361856832 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.4608 - categorical_accuracy: 0.3718Source path =  ./Project_data/val ; batch size = 10

Epoch 00001: saving model to Gesture_recog_temptest_2021-02-0817_20_52.947547/model-00001-1.46076-0.37179-2.07324-0.23000.h5
67/67 [==============================] - 84s 1s/step - loss: 1.4608 - categorical_accuracy: 0.3718 - val_loss: 2.0732 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.4613 - categorical_accuracy: 0.3582
Epoch 00002: saving model to Gesture_recog_temptest_2021-02-0817_20_52.947547/model-00002-1.46130-0.35821-1.95097-0.26000.h5
67/67 [==============================] - 29s 431ms/step - loss: 1.4613 - categorical_accuracy: 0.3582 - val_loss: 1.9510 - val_categorical_accuracy: 0.2600 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.3677 - categorical_accuracy

# Experimenting with higher number of epochs (30 vs 10).

In [ ]:
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - ETA: 0s - loss: 1.3933 - categorical_accuracy: 0.4005Source path =  ./Project_data/val ; batch size = 10

Epoch 00001: saving model to Gesture_recog_temptest_2021-02-0818_47_54.308796/model-00001-1.39331-0.40045-2.31297-0.23000.h5
67/67 [==============================] - 101s 2s/step - loss: 1.3933 - categorical_accuracy: 0.4005 - val_loss: 2.3130 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/30
67/67 [==============================] - ETA: 0s - loss: 1.3705 - categorical_accuracy: 0.4179
Epoch 00002: saving model to Gesture_recog_temptest_2021-02-0818_47_54.308796/model-00002-1.37046-0.41791-1.80048-0.24000.h5
67/67 [==============================] - 31s 458ms/step - loss: 1.3705 - categorical_accuracy: 0.4179 - val_loss: 1.8005 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 3/30
67/67 [==============================] - ETA: 0s - loss: 1.4183 - categorical_accurac

In [ ]:
num_epochs = 60
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/train ; batch size = 10
Epoch 1/60
67/67 [==============================] - ETA: 0s - loss: 1.3975 - categorical_accuracy: 0.4035Source path =  ./Project_data/val ; batch size = 10

Epoch 00001: saving model to Gesture_recog_temptest_2021-02-0910_48_46.983282/model-00001-1.39752-0.40347-2.01449-0.24000.h5
67/67 [==============================] - 136s 2s/step - loss: 1.3975 - categorical_accuracy: 0.4035 - val_loss: 2.0145 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 2/60
67/67 [==============================] - ETA: 0s - loss: 1.3573 - categorical_accuracy: 0.4129
Epoch 00002: saving model to Gesture_recog_temptest_2021-02-0910_48_46.983282/model-00002-1.35731-0.41294-1.40966-0.41000.h5
67/67 [==============================] - 30s 453ms/step - loss: 1.3573 - categorical_accuracy: 0.4129 - val_loss: 1.4097 - val_categorical_accuracy: 0.4100 - lr: 0.0010
Epoch 3/60
67/67 [==============================] - ETA: 0s - loss: 1.3667 - categorical_accurac

### Model performance plateaus even with 6x higher epochs at around 76% validation accuracy for LSTM with GRU.

# Choosing model 6 with batch size 40 (model 6c in write up) given similar performance with 35% less trainable parameters, i.e. less training required and also requires lower memory to live. However, there is slight tradeoff in terms of overfitting.